In [17]:
import numpy as np
import pandas as pd
import re
import yfinance as yf
import pandas_market_calendars as mcal

Loading data

In [2]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [3]:
df = df.loc[df['TYPE'] == 'MERGED']

Extracting Time and Data

In [4]:
df["Time"] = df.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df["DATE"] = df.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

Cleaning and organizing

In [6]:
df['DATE'] = pd.to_datetime(
                          df['DATE'],
                          format='%Y-%m-%d')

In [7]:
df = df.sort_values(by="DATE")

In [8]:
df.columns = ['DATE', 'ID', 'STOCK',"SECTOR","TYPE","SCORE","TIME"]

In [9]:
df['SCORE']=df['SCORE'].replace(0, np.nan)

Adding Returns

In [10]:
list_of_companies = df["STOCK"].unique()
list_of_dates = df["DATE"].unique()

In [11]:
start_date = df.DATE.min()
end_date = df.DATE.max()

In [12]:
final_date = end_date + pd.DateOffset(months=12)
print(final_date)

2005-07-14 00:00:00


In [ ]:
Columns_names = ['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn']

In [13]:
# extract information about historical prices of stocks for given time period and stock
def get_history_data(ticker_symbol, start_date, end_date):
    tickerData = yf.Ticker(ticker_symbol)
    tickerDf = tickerData.history(period='1d', start=start_date, end=end_date)
    tickerDf["STOCK"] = ticker_symbol

    return tickerDf

In [14]:
# find the trading day which is the same as current date or the closest future day
def get_closest_trading_day(day,trading_days):
    trading_days = trading_days[trading_days>=pd.to_datetime(day, utc=True)]
    return trading_days.sort_values()[0]

In [18]:
# get list of all trading dates between and start and end date on particular market
def get_traning_days(start_date, end_date,stock_market_name):
    nyse = mcal.get_calendar(stock_market_name)
    valid_days = nyse.valid_days(start_date=start_date, end_date=end_date)
    return valid_days

In [ ]:
#extracting close price on given data and formating it in case of non such data

def get_return_on_date(date,df):
  try:
    x = df.loc[date]
    x = float(x['Close'])
  except:
    x = 0.0
  return x